In [3]:
import pickle
import os

with open(os.path.join("results", "glass", f'results_0.pickle'), 'rb') as f:
    results = pickle.load(f)

results

{'baseline_knn_time': array([0.1540075, 0.0070467, 0.0059965]),
 'baseline_knn_acc': array([0.62790698, 0.69767442, 0.58139535]),
 'baseline_time': array([0.0013885, 0.0009983, 0.0085672, 0.0037015]),
 'baseline_acc': array([0.58139535, 0.53488372, 0.6744186 , 0.72093023]),
 'smart_time': array([[[2.3594546, 0.0590264],
         [0.0597788, 0.0608105],
         [0.0633466, 0.0617818]],
 
        [[0.0539741, 0.0560436],
         [0.0561274, 0.0598776],
         [0.0579494, 0.0598743]],
 
        [[0.0562593, 0.0647282],
         [0.0617295, 0.0715498],
         [0.0676939, 0.0727452]],
 
        [[0.0645766, 0.0715691],
         [0.0708678, 0.0721596],
         [0.0748834, 0.0767115]]]),
 'smart_acc': array([[[0.60465116, 0.6744186 ],
         [0.69767442, 0.6744186 ],
         [0.69767442, 0.69767442]],
 
        [[0.60465116, 0.62790698],
         [0.6744186 , 0.69767442],
         [0.62790698, 0.65116279]],
 
        [[0.62790698, 0.69767442],
         [0.6744186 , 0.6744186 ],
    